In [1]:
import configparser
from datetime import datetime
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import pandas as pd

In [28]:
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

In [29]:
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']["AWS_ACCESS_KEY_ID"]
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']["AWS_SECRET_ACCESS_KEY"]

In [30]:
spark = SparkSession \
.builder \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
.getOrCreate()

In [31]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://misho-udacity-bucket/datalake/"

# Getting data

In [32]:
df = spark.read.json("s3a://{}:{}@udacity-dend/song_data/A/B/C/TRABCEI128F424C983.json".format(os.environ['AWS_ACCESS_KEY_ID'],os.environ['AWS_SECRET_ACCESS_KEY']))

In [33]:
df.limit(1).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARJIE2Y1187B994AB7,None,,None,Line Renaud,152.92036,1,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,0


In [22]:
song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

In [23]:
df_song = df.select(*song_columns)
df_artist = df.select(*artist_columns)

In [21]:
df_song.show()

+------------------+-------------------+------------------+----+---------+
|           song_id|              title|         artist_id|year| duration|
+------------------+-------------------+------------------+----+---------+
|SOUPIRU12A6D4FA1E1|Der Kleine Dompfaff|ARJIE2Y1187B994AB7|   0|152.92036|
+------------------+-------------------+------------------+----+---------+



In [24]:
df_artist.show()

+------------------+-----------+---------------+---------------+----------------+
|         artist_id|artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+-----------+---------------+---------------+----------------+
|ARJIE2Y1187B994AB7|Line Renaud|               |           null|            null|
+------------------+-----------+---------------+---------------+----------------+



### Partition

In [37]:
df_song_part = df_song.write.partitionBy("year", "artist_id").parquet('output/songs/')

# Write data

In [35]:
df_song.write.parquet(output_data+'/songs/')